# Day 4
## So far
- Text cleaning
- Convert text to document term matrix
- Properties of document term matrix
- Text classification (naive bayes algorithm)
- Sentiment Analysis
    - Supervised algorithm
    - VADER
- Text clustering/ Topic Modelling
- Basics of web scraping
- Text summarization
- Chat bot introduction
## Topics
- Case study: Reviews from Zomato website
    - Web scraping
    - Exploratory Data Analysis
    - Sentiment Analysis
    - Sentiment classifier
    - Recommendation system
    - Topic modelling
- Twitter web scraping
- Bigrams/Trigrams, word similarity, Lemmatization
- Chatbot

## Case study/ Project: Zomato Reviews

In [2]:
# !pip install selenium

In [32]:
from selenium import webdriver
import pandas as pd

In [67]:
browser = webdriver.Firefox()

In [23]:
url = 'https://www.zomato.com/hyderabad/great-food-no-bull'
browser.get(url)

In [61]:
res_tags = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')
len(res_tags)

0

In [36]:
restaurants = pd.DataFrame()
for res in res_tags:
    title = res.find_element_by_css_selector('div.res_title').text
    loc_sel = 'div.nowrap.grey-text.fontsize5.ttupper'
    loc = res.find_element_by_css_selector(loc_sel).text
    food_type = res.find_elements_by_css_selector('div.nowrap.grey-text')
    food_type = food_type[1].text
    res_id = res.get_attribute('data-res-id')
    rating = res.get_attribute('data-rating')
    url = res.find_element_by_css_selector('a[data-link-type]')
    url = url.get_attribute('href')
    curr_res = {'title': title, 'location': loc, 'food_type': food_type,
                'res_id': res_id, 'rating': rating, 'url': url}
    restaurants = restaurants.append(curr_res, ignore_index=True)

In [39]:
#restaurants.head()

In [42]:
url = restaurants.loc[0, 'url']
browser.get(url)

In [59]:
import time
while True:
    try:
        browser.find_element_by_css_selector('div.res-page-load-more').click()
        time.sleep(2)
    except:
        break

In [62]:
sel = 'div.ui.segment.clearfix.brtop'
#res_id = restaurants.loc[0, 'res_id']
df_reviews = pd.DataFrame()
reviews_tags = browser.find_elements_by_css_selector(sel)
for review in reviews_tags:
    rev_name = review.find_element_by_css_selector('a[data-entity_id]').text
    rev_id = review.find_element_by_css_selector(
        'a[data-entity_id]').get_attribute('data-entity_id')
    sel = 'span.grey-text.fontsize5.nowrap'
    rev_count = review.find_element_by_css_selector(sel).text
    date = review.find_element_by_css_selector('time').get_attribute('datetime')
    rat_sel = 'div.ttupper.fs12px.left.bold.zdhl2.tooltip'
    rating = review.find_element_by_css_selector(
        rat_sel).get_attribute('aria-label')
    text = review.find_element_by_css_selector('div.rev-text').text
    df_reviews = df_reviews.append({
        'rev_name': rev_name,
        'rev_id': rev_id,
        'rev_count': rev_count,
        'date': date,
        'rating': rating,
        'text': text
    }, ignore_index=True)
df_reviews.shape

(144, 6)

In [64]:
def get_restaurant_reviews(browser, url):
    browser.get(url)
    while True:
        try:
            browser.find_element_by_css_selector('div.res-page-load-more').click()
            time.sleep(2)
        except:
            break
    df_reviews = pd.DataFrame()
    sel = 'div.ui.segment.clearfix.brtop'
    reviews_tags = browser.find_elements_by_css_selector(sel)
    for review in reviews_tags:
        rev_name = review.find_element_by_css_selector('a[data-entity_id]').text
        rev_id = review.find_element_by_css_selector(
            'a[data-entity_id]').get_attribute('data-entity_id')
        sel = 'span.grey-text.fontsize5.nowrap'
        rev_count = review.find_element_by_css_selector(sel).text
        date = review.find_element_by_css_selector('time').get_attribute('datetime')
        rat_sel = 'div.ttupper.fs12px.left.bold.zdhl2.tooltip'
        rating = review.find_element_by_css_selector(
            rat_sel).get_attribute('aria-label')
        text = review.find_element_by_css_selector('div.rev-text').text
        df_reviews = df_reviews.append({
            'rev_name': rev_name,
            'rev_id': rev_id,
            'rev_count': rev_count,
            'date': date,
            'rating': rating,
            'text': text
        }, ignore_index=True)
    return df_reviews

In [68]:
all_res_reviews = pd.DataFrame()
for index, res in restaurants.head(2).iterrows():
    res_reviews = get_restaurant_reviews(browser, res['url'])
    res_reviews['res_name'] = res['title']
    res_reviews['res_id'] = res['res_id']
    all_res_reviews = all_res_reviews.append(res_reviews, ignore_index=True)
    all_res_reviews.shape